# 파이썬 추천 시스템 심화과정

추천 시스템은 주로 매우 큰 데이터 셋에 기반하고 특별한 방법에 따라 잘 관리되어야 한다. 이 때문에 우리는 이 토픽에 대해 프로젝트를 진행하지는 않을 거싱다. 대신 우리는 Movie Lens 데이터 셋을 통해 Python으로 추천 시스템을 만들 것이다.

*주의 : 추천 시스템 뒤에 있는 수학적 지식은 선형대수학이 꽤 많이 들어간다.* <u>라고는 하는데 쫄지맙시다. </u>
___

## 사용되는 방법들

추천 시스템의 가장 흔한 두 가지 타입을 뽑자면 **컨텐츠-기반** 과 **협업 필터링 (CF) ** 이다.

Two most common types of recommender systems are **Content-Based** and **Collaborative Filtering (CF)**

* 협업 필터링은 사용자들의 아이템들에 대한 태도에 기초하여 추천을 한다. 즉, "다수의 지혜"를 빌려 아이템을 추천하는 것이다.
* 컨텐츠-기반 추천 시스템들은 아이템들의 특성에 집중하여 아이템들의 유사도에 기초하여 추천을 해 주는 방식이다.

## 협업 필터링(Collaborative Filtering)

일반적으로, 협업 필터링 (CF)는 컨텐츠-기반 시스템보다 더 많이 사용된다. 왜냐하면 보통 보다 나은 결과를 도출하고 상대적으로 전체적인 구현이 이해하기 쉽기 때문이다. CF 알고리즘은 피처 학습을 스스로 하는 능력이 있다. 즉 무슨 피처를 사용할지 학습하는 능력이 있다는 것이다.

협업 필터링은 **기억-기반 협업 필터링** 과 **모델-기반 협업 필터링** 으로 나뉘어진다.

이 튜토리얼에서는, 우리는 모델-기반 CF를 특이값 분해(SVD, Singular Value Decomposition)로 구현하고 기억-기반 CF를 코사인 유사도를 통해 구현할 것이다.


## 사용 데이터

우리는 유명한 무비렌즈 데이터셋을 사용할 것이다. 이는 가장 많이되는 데이터셋중 하나이며 추천 시스템을 구현하고 테스트하는데 많이 사용된다. 이는 10만개 이상의 영화 평점을 943명의 유저에게 1682개의 영화에 대해 한 것이다.

데이터 셋은 [여기](http://files.grouplens.org/datasets/movielens/ml-100k.zip)에서 다운 받을 수 있다. 혹은 폴더에 있는 u.data 파일을 사용해라.

____
## 시작해 보자

먼저 필요한 라이브러리를 불러오자:

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

모든 데이터셋을 갖고 있는 **u.data** 파일을 읽을 수 있다. 이 데이터셋에 대한 간략한 요약은 다음과 같다 [here](http://files.grouplens.org/datasets/movielens/ml-100k-README.txt).

어떻게 Tab 분리 인수 (separator argument)가 작동하는지 살펴보자.

In [2]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)

데이터를 잠시 살펴보자

In [3]:
df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


item_id만 있고 영화 제목이 없는 것에 주목하자. 우리는 Movie_ID_Titles csv 파일을 이용하여 영화 제목을 가지고 와서 merge를 사용하여 이 데이터 프레임에 붙일 수 있다:

In [4]:
movie_titles = pd.read_csv("Movie_Id_Titles")
movie_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


이제 데이터프레임들을 머지를 해 보자:

In [5]:
df = pd.merge(df,movie_titles,on='item_id')
df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


유저의 수와 영화의 수를 살펴보자.

In [6]:
n_users = df.user_id.nunique()
n_items = df.item_id.nunique()

print ('Num. of Users: ' + str(n_users))
print ('Num of Movies: ' + str(n_items))

Num. of Users: 944
Num of Movies: 1682


## 학습 셋과 테스트 셋 나누기

추천시스템은 자연적으로 매우 평가하기가 어렵다, 하지만 이 튜토리얼을 통해서 어떻게 이를 평가하는지 보여줄 것이다. 이를 위해서, 우리는 먼저 데이터를 두 셋으로 나눌 것이다. 하지만, 우리는 일반적인 X_train, X_test, y_train, y_test 나누기는 하지 않을 것이다. 대신, 우리는 데이터를 그저 두 분류로 나눌 것이다:

In [7]:
from sklearn.cross_validation import train_test_split
train_data, test_data = train_test_split(df, test_size = 0.25)
train_data.head()

,user_id,item_id,rating,timestamp,title
18391,700,96,4,884494310,Terminator 2: Judgment Day (1991)
7429,811,304,5,886377311,Fly Away Home (1996)
92155,276,1036,2,889174870,Drop Dead Fred (1991)
7196,318,796,3,884496500,Speechless (1994)
48396,896,325,1,887157732,Crash (1996)


## 기억-기반 협업 필터링

기억-기반 협업 필터링은 다시 두가지 메인 부분으로 나뉠 수 있다: **user-item 필터링** 과 **item-item 필터링**.

*유저-아이템 필터링* 은 하나의 유저를 선택하여, 평점에 기반하여 비슷한 유저를 찾는다. 그리고 그 유사한 유저가 좋아했던 아이템들을 추천해 준다.

반면, *item-item 필터링* 은 아이템 하나를 선택하여, 그 아이템을 좋아한 유저들을 찾는다. 그 유저들 혹은 유사한 유저들이 좋아했던 아이템들을 찾는다. 이는 아이템들을 가지고 다른 아이템을 추천하는 방식이다.

* *아이템-아이템 협업 필터링* : "이 아이템을 좋아한 유저들은 이것도 …”
* *유저-아이템 협업 필터링* : "당신과 비슷한 유저들은 이것도 …”

두가지 경우 모두, 당신은 유저-아이템 메트릭스를 전체 데이터 셋으로부터 만들어야 한다.

데이터를 테스트 셋과 학습 셋으로 나누었기 때문에, 우리는 2개의 ``[943 x 1682]`` 행렬을 만들 수 있다 (모든 유저들과 모든 영화의 곱).

학습 행렬은 75%의 별점들을 갖고 있고, 테스트 행렬은 25%의 별점들을 갖고 있다.

유저-아이템 행렬의 예:
<img class="aligncenter size-thumbnail img-responsive" src="http://s33.postimg.org/ay0ty90fj/BLOG_CCA_8.png" alt="blog8"/>

유저-아이템 행렬을 구성한 후, 당신은 유사도를 계산하고 유사도 행렬을 만들 수 있다.

*아이템-아이템 협업 필터링* 안의 아이템 사이의 유사도 값은 두 아이템 모두를 평가한 모든 유저들을 관측하여 계산한다.

The similarity values between items in *Item-Item Collaborative Filtering* are measured by observing all the users who have rated both items.  

<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="http://s33.postimg.org/i522ma83z/BLOG_CCA_10.png"/>

*유저-아이템 협업 필터링*

유저 사이의 *유저-아이템 협업 필터링*의 유사도 값은 두 사용자가 모두 평가한 모든 아이템을 관측하여 계산된다.

For *User-Item Collaborative Filtering* the similarity values between users are measured by observing all the items that are rated by both users.

<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="http://s33.postimg.org/mlh3z3z4f/BLOG_CCA_11.png"/>

추천 시스템에서 주로 사용되는 거리 계산법은 *코사인 유사도* 이다. 여기서 평점들은 벡터로서 ``n``-차원의 공간에 있고 유사도는 이 벡터들 사이의 각도에 기반해서 계산된다. 유저 *a*와 *m*사이의 코사인 유사도는 다음 공식에 의해 계산될 수 있다. 유저들의 벡터 *$u_k$*와 *$u_a$*의 내적을 계산하고 이를 벡터들의 유클리디안 길이의 곱으로 나누어 준다.

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(u_k,u_a)=\frac{u_k&space;\cdot&space;u_a&space;}{&space;\left&space;\|&space;u_k&space;\right&space;\|&space;\left&space;\|&space;u_a&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{k,m}x_{a,m}}{\sqrt{\sum&space;x_{k,m}^2\sum&space;x_{a,m}^2}}"/>

아이템들 *m*과 *b* 사이의 유사도를 계산하기 위해서는 이 공식을 사용한다:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(i_m,i_b)=\frac{i_m&space;\cdot&space;i_b&space;}{&space;\left&space;\|&space;i_m&space;\right&space;\|&space;\left&space;\|&space;i_b&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{a,m}x_{a,b}}{\sqrt{\sum&space;x_{a,m}^2\sum&space;x_{a,b}^2}}
"/>

먼저 유저-아이템 행렬을 만들어야 한다. 우리는 테스트 데이터와 학습 데이터 둘을 갖고 있기 때문에 행렬도 두개를 만들어야 한다.

In [8]:
# itertuples를 통해서 dataframe을 튜플들 형태로 값을 받아올 수 있다.
for line in train_data.head().itertuples():
    print (line)

Pandas(Index=18391, user_id=700, item_id=96, rating=4, timestamp=884494310, title='Terminator 2: Judgment Day (1991)')
Pandas(Index=7429, user_id=811, item_id=304, rating=5, timestamp=886377311, title='Fly Away Home (1996)')
Pandas(Index=92155, user_id=276, item_id=1036, rating=2, timestamp=889174870, title='Drop Dead Fred (1991)')
Pandas(Index=7196, user_id=318, item_id=796, rating=3, timestamp=884496500, title='Speechless (1994)')
Pandas(Index=48396, user_id=896, item_id=325, rating=1, timestamp=887157732, title='Crash (1996)')


In [9]:
for line in train_data.head().itertuples():
    print (line[0], line[1], line[2], line[3])

18391 700 96 4
7429 811 304 5
92155 276 1036 2
7196 318 796 3
48396 896 325 1


In [10]:
# 두 개의 유저-아이템 행렬을 만든다. 하나는 학습을 위한 것이고 하나는 테스트를 위한 것이다.
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  # train_data_matrix 행렬에 [user_id-1, item_id-1] 위치에 평점 정보를 입력

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [11]:
train_data_matrix

array([[ 0.,  3.,  4., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  5.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [12]:
train_data_matrix.shape

(944, 1682)

여기서 [pairwise_distances](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html)함수를 sklearn에서 사용하여 코사인 유사도를 계산한다. 모든 값은 0과 1사이에 존재하는 것을 기억하자.

In [13]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(np.array([[1,1], [1,0]]), metric = 'cosine')

In [14]:
1 - user_similarity  # 이렇게 하면 당신이 상상하는 그 유사도

array([[ 1.        ,  0.70710678],
       [ 0.70710678,  1.        ]])

In [15]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine') # train_data_matrix.T 라는 것에 주목하자.

In [16]:
user_similarity.shape

(944, 944)

In [17]:
item_similarity.shape

(1682, 1682)

In [18]:
user_similarity

array([[ -4.44089210e-16,   8.97997787e-01,   9.67526132e-01, ...,
          8.53689785e-01,   6.80990223e-01,   8.64886795e-01],
       [  8.97997787e-01,   0.00000000e+00,   8.77546079e-01, ...,
          8.63568513e-01,   9.15173648e-01,   8.68528444e-01],
       [  9.67526132e-01,   8.77546079e-01,   3.33066907e-16, ...,
          9.04451284e-01,   9.79449991e-01,   1.00000000e+00],
       ..., 
       [  8.53689785e-01,   8.63568513e-01,   9.04451284e-01, ...,
          0.00000000e+00,   8.89155736e-01,   8.99069574e-01],
       [  6.80990223e-01,   9.15173648e-01,   9.79449991e-01, ...,
          8.89155736e-01,   1.11022302e-16,   8.65413667e-01],
       [  8.64886795e-01,   8.68528444e-01,   1.00000000e+00, ...,
          8.99069574e-01,   8.65413667e-01,   2.22044605e-16]])

In [19]:
1 - user_similarity

array([[ 1.        ,  0.10200221,  0.03247387, ...,  0.14631022,
         0.31900978,  0.1351132 ],
       [ 0.10200221,  1.        ,  0.12245392, ...,  0.13643149,
         0.08482635,  0.13147156],
       [ 0.03247387,  0.12245392,  1.        , ...,  0.09554872,
         0.02055001,  0.        ],
       ..., 
       [ 0.14631022,  0.13643149,  0.09554872, ...,  1.        ,
         0.11084426,  0.10093043],
       [ 0.31900978,  0.08482635,  0.02055001, ...,  0.11084426,
         1.        ,  0.13458633],
       [ 0.1351132 ,  0.13147156,  0.        , ...,  0.10093043,
         0.13458633,  1.        ]])

In [20]:
np.array([[2,2],[2,2]]) - 1

array([[1, 1],
       [1, 1]])

이제 예측을 해야 한다. 이미 우리는 유사도 행렬이 있다: `유저_유사도` 와 `아이템_유사도` 행렬이 위에서 만들어졌다. 

이를 통해서 유저-기반 CF 공식을 적용하여 예측을 할 수 있다:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\bar{x}_{k}&space;&plus;&space;\frac{\sum\limits_{u_a}&space;sim_u(u_k,&space;u_a)&space;(x_{a,m}&space;-&space;\bar{x_{u_a}})}{\sum\limits_{u_a}|sim_u(u_k,&space;u_a)|}"/>

이제 유저들 *k*와 *a*의 유사도를 유사한 유저 *a*(그 유저의 평균 평점으로 수정된)의 평점들이 곱해진 가중치라고 볼 수 있다. 이제 이것을 정규화(normalize)하여서 평점들이 1과 5사이에 머물도록 해야 한다. 그리고 마지막 단계로서, 당신이 예측하려고 하는 유저의 평균 평점을 합한다. 
You can look at the similarity between users *k* and *a* as weights that are multiplied by the ratings of a similar user *a* (corrected for the average rating of that user). You will need to normalize it so that the ratings stay between 1 and 5 and, as a final step, sum the average ratings for the user that you are trying to predict. 

이 생각의 핵심은 어떠한 유저는 모든 영화에 대해서 높거나 낮은 평점을 줄 것이라는 것이다. 유저들이 주는 평점에서의 상대적인 차이가 절대값보다 더 중요하다. 예를 들어보자: 유저 *k* 가 그가 가장 좋아하는 영화에 대해서 4점을 주고 다른 모든 좋은 영화에 대해 3점을 주었다고 하자. 이제 다른 유저 *t* 가 자기가 좋아하는 영화들에 대해서 5점을 주고, 그닥 흥미롭지 않은 영화에 대해서는 3점을 주었다고 하자. 이 두 사용자는 매우 비슷한 취향을 지녔지만 평점을 다르게 매긴다.

아이템-베이스의 CF에 기반한 예측을 할때, 질의 유저 스스로가 이미 예측에 익숙하기 때문에 유저들의 평균 별점들을 수정할 필요가 없다. 

When making a prediction for item-based CF you don't need to correct for users average rating since query user itself is used to do predictions.

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\frac{\sum\limits_{i_b}&space;sim_i(i_m,&space;i_b)&space;(x_{k,b})&space;}{\sum\limits_{i_b}|sim_i(i_m,&space;i_b)|}"/>

In [21]:
np.array([[1,2,3],[4,5,6],[7,8,9]]) # ratings 예제

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [22]:
np.array([[1,2,3],[4,5,6],[7,8,9]]).mean(axis=1) # ratings를 axis=1로 하여 평균

array([ 2.,  5.,  8.])

In [23]:
np.array([[1,2,3],[4,5,6],[7,8,9]]).mean(axis=0)

array([ 4.,  5.,  6.])

In [24]:
np.array([[1,2,3],[4,5,6],[7,8,9]]).mean(axis=1)[:, np.newaxis] # 여기에 축을 추가해 주면

array([[ 2.],
       [ 5.],
       [ 8.]])

In [25]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [26]:
train_data_matrix

array([[ 0.,  3.,  4., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  5.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [27]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [28]:
item_prediction.shape

(944, 1682)

In [29]:
user_prediction.shape

(944, 1682)

### 평가
많은 평가 방법이 있겠지만, 예측한 평점 결과의 정확도를 측정하기 위해 가장 유명한 평가 방법은 *Root Mean Squared Error (RMSE)*. 방법이다.
<img src="https://latex.codecogs.com/gif.latex?RMSE&space;=\sqrt{\frac{1}{N}&space;\sum&space;(x_i&space;-\hat{x_i})^2}" title="RMSE =\sqrt{\frac{1}{N} \sum (x_i -\hat{x_i})^2}" />

[mean square error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) (MSE) 는 `sklearn`의 함수로부터 사용할 수 있다. 여기서 RMSE는 그저 MSE에 제곱근(루트)가 들어간 것이다. 다른 평가 방법들에 대해 알아보려면 이 문서를 참고해라. [문서](http://research.microsoft.com/pubs/115396/EvaluationMetrics.TR.pdf).

테스트 데이터 셋 안에 정보가 있는 예상된 평점들만 고려해야 하기 때문에, `prediction[ground_truth.nonzero()]`를 사용하여 예상 행렬 안의 해당되지 않은 모든 원소들을 제거한다.

In [30]:
np.array([0,1,0,1,0]).nonzero()

(array([1, 3], dtype=int64),)

In [31]:
np.array([0,1,0,0]).nonzero

<function ndarray.nonzero>

In [32]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [33]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.1234028214938365
Item-based CF RMSE: 3.4499186909844686


기억-기반 알고리즘들은 구현도 쉽고 예측의 품질도 준수하다. 기억-기반 CF의 단점은 실무에서 방대한 데이터를 접할때 문제가 발생할 수 있고 처음 보는 아이템이나 새 유저에 대해서는 추천을 하기 힘들다는 단점이 있다. 모델-기반 CF는 방대한 데이터에도 적용 가능하고 보다 원소가 더 희소(higher sparsity level)할 경우에도 기억-기반 모델들보다 더 잘 작동할 수 있다. 하지만 이 모델 또한 신규 유저나 아이템이 전혀 정보가 없을때에는 문제가 생긴다. 기억 기반 협업 필터링에 대해서는 Ethan Rosenthal의 [블로그](http://blog.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/) 에 잘 정리되어 있다.

# 모델-기반의 협업 필터링

모델-기반의 협업 필터링은 **행렬 분해(matrix factorization, mf)** 에 기반하고 있다. 이는 주로 잠재적 변수 분해(latent variable decomposition) 혹은 차원 축소 등의 비지도적 학습 방법을 적용할때 많이 나타난다. 행렬 분해는 보다 큰 데이터나 요소가 더 희소한 데이터에 대해서 기억-기반 모델보다 더 잘 작동할 수 있는 추천 시스템에 대해 잘 사용된다. 행렬 분해의 목표는 유저나 아이템의 잠재적 선호나 특성을 알려진 평점 정보로부터(평점들의 특징을 잘 묘사할 수 있는 피처들을) 학습하여 알지 못하는 평점들을 유저의 잠재적 피처와 아이템의 내적 곱을 통해서 예측해 낸다.

차원이 높고 데이터가 희소한 행렬에 대해서 행렬 분해를 적용하면, 유저-아이템 행렬을 보다 낮은 차원의 행렬 구조로 재구축하여 낮은 두 차원의 행렬 곱으로 나타낼 수 있다. 여기서 행들은 특징을 가진 잠재 벡터(the latent vector)를 포함한다. 당신은 낮은 차원의 행렬을 곱하여 이 행렬을 원래의 행렬에 최대한 맞게 피팅시킬 수 있다. 이는 원래 행렬에 비었던 부분들을 채워 줄 것이다.

무비렌즈 데이터셋의 희소도(sparsity, 듬성듬성함)을 계산해 보자:

In [34]:
len(df) # 샘플의 수

100003

In [35]:
n_users * n_items   # unique한 유저의 수 * unique 한 아이템의 수

1587808

In [36]:
len(df)/float(n_users*n_items)

0.06298179628771237

In [37]:
sparsity=round(1.0-len(df)/float(n_users*n_items),3)
print('The sparsity level of MovieLens100K is ' +  str(sparsity*100) + '%' + ' ..빈게 많다능..')

The sparsity level of MovieLens100K is 93.7% ..빈게 많다능..


잠재하고 있는 유저와 아이템의 선호도에 대한 예를 들어 보자: 무비렌즈 데이터 셋이 다음과 같은 정보를 가지고 있다고 하자: _(user id, age, location, gender, movie id, director, actor, language, year, rating)_. 행렬 분해를 적용함으로서, 모델은 유저의 중요한 피처들이  _age group (under 10, 10-18, 18-30, 30-90)_, _location_ and _gender_ 이라는 것을 학습할 것이고 영화들에 관해 중요한 피처들은 _decade_, _director_ and _actor_ 이 가장 중요한 정보라는 것을 학습할 것이다. 저장한 정보를 보면, _decade_ 같은 피처들은 존재하지 않지만 모델을 스스로 학습할 수 있다. 가장 중요한 측면은 이 모델-기반 CF모델은 오직 (user_id, movie_id, rating) 데이터를 사용하여 이 잠재 피처들을 학습할 것이라는 것이다. 만약 가능한 데이터가 매우 적다면 CF 모델은 예측을 매우 부실하게 할 것이고, 잠재 피처들을 학습하기가 더욱 어려울 것이다.

별점과 컨텐츠 피처들을 모두 사용하는 모델은 **하이브리드 추천 시스템(Hybrid Recommender Systems)** 라 불리는데 이는 협업 필터링과 컨텐츠-기반 모델의 합이다. 혼합 추천 시스템은 주로 협업 필터링 혹은 컨텐츠-기반 모델 각각보다 성능이 좋다: 이들은 유저나 아이템의 평가 정보가 없더라도 유저나 아이템의 정보를 사용하여 예측을 할 수 있다.

### 특이값 분해(Singular Value Decomposition, SVD)

잘 알려진 행렬 분해 방법 중 하나는 **특이값 분해 (SVD)** 이다. 협업 필터링은 행렬 `X`를 특이값 분해를 이용해 근사치를 구함으로서 만들어진다. Netflix Prize 대회의 승자는 SVD 행렬 분해를 사용하여 추천을 하였다. 이상의 정보에 대해서 다음의 기사를 읽을 것을 추천한다: [Netflix Recommendations: Beyond the 5 stars](http://techblog.netflix.com/2012/04/netflix-recommendations-beyond-5-stars.html) 과 [Netflix Prize and SVD](http://buzzard.ups.edu/courses/2014spring/420projects/math420-UPS-spring-2014-gower-netflix-SVD.pdf).

잘 알려진 등식은 다음과 같이 나타난다:
<img src="https://latex.codecogs.com/gif.latex?X=USV^T" title="X=USV^T" />

주어진 `m x n` 행렬 `X`:
* *`U`* 는 *`(m x r)`* 직교 행렬
* *`S`* 는 *`(r x r)`* 음의 실수가 주 대각성분에 없는 대각 행렬
* *$V^T$* 는 *`(r x n)`* 직교 행렬

주 대각성분 `S`는 *특이값 `X`* 라고 불린다

행렬 *`X`* 은 *`U`*, *`S`* 그리고 *`V`* 로 인수분해 될 수 있다. *`U`* 행렬은 숨겨진 피처 공간에서 해당하는 유저들에 피처 벡터들을 나타내고 *`V`* 행렬은 숨겨진 피처 공간에서 해당하는 아이템들에 대한 피처 벡터들을 의미한다. <img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="http://s33.postimg.org/kwgsb5g1b/BLOG_CCA_5.png"/>

이제 *`U`*, *`S`* 그리고 *`V^T`* 에 대해서 내적(dot product)을 계산하여 예측을 할 수 있다.

<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="http://s33.postimg.org/ch9lcm6pb/BLOG_CCA_4.png"/>

In [38]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
print('User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix)))

User-based CF MSE: 2.7233130061100517


오직 몇개의 알려진 입력값만 대입해서는 오버피팅이 되기 쉽다. SVD는 매우 느리고 연산적으로 비싼 작업이다. 최근에는 제곱된 에러의 값을 줄이기 위해 alternating least square(ALS) 혹은 확률적 경사 하강법(stochastic gradient descent, SGD) 를 사용하고 정규화 조건을 적용하여 오버피팅을 방지한다. Alternating least square 방식이나 확률적 경사 하강법을 CF에 적용하는 방법은 다음 튜토리얼들에서 설명될 것이다. (정말?)

요약하자면:

* 어떻게 간단하게 두 가지의 **협업 필터링** 방법들(기억-기반, 모델-기반)을 구현하는지 살펴보았다.
* **기억-기반 모델들** 은 아이템들 혹은 유저들의 유사도에 기반하는데, 여기서는 코사인 유사도를 적용하였다.
* **모델-기반 CF **는 행렬 분해에 기초하는데 특이값 분해(SVD)를 사용하여 행렬을 분해하였다.
* 정보가 없는 경우에도 잘 작동하는 추천 시스템을 설계하는 것은 여전히 어려운 문제이다.(새로운 유저들이나 아이템들에 대해 매우 적은 정보만 있을 경우에) 표준 협업 필터링 방법은 이러한 경우에 잘 작동하지 못한다.

## 더 알아보기

추천 시스템 분석에 대해 더 알아보려면, 아래의 데이터 셋들을 확인해 봐라. 주의: 이 데이터들은 대부분의 경우에 꽤 크다, 링크가 동작하지 않을 수도 있지만 대부분은 여전히 잘 작동할 것이다. 안되면 구글링을 사용해서 데이터를 직접 찾아보자!


**영화 추천:**

MovieLens - Movie Recommendation Data Sets http://www.grouplens.org/node/73

Yahoo! - Movie, Music, and Images Ratings Data Sets http://webscope.sandbox.yahoo.com/catalog.php?datatype=r

Jester - Movie Ratings Data Sets (Collaborative Filtering Dataset) http://www.ieor.berkeley.edu/~goldberg/jester-data/

Cornell University - Movie-review data for use in sentiment-analysis experiments http://www.cs.cornell.edu/people/pabo/movie-review-data/

**음악 추천:**

Last.fm - Music Recommendation Data Sets http://www.dtic.upf.edu/~ocelma/MusicRecommendationDataset/index.html

Yahoo! - Movie, Music, and Images Ratings Data Sets http://webscope.sandbox.yahoo.com/catalog.php?datatype=r

Audioscrobbler - Music Recommendation Data Sets http://www-etud.iro.umontreal.ca/~bergstrj/audioscrobbler_data.html

Amazon - Audio CD recommendations http://131.193.40.52/data/

**책 추천:**

Institut für Informatik, Universität Freiburg - Book Ratings Data Sets http://www.informatik.uni-freiburg.de/~cziegler/BX/
Food Recommendation:

Chicago Entree - Food Ratings Data Sets http://archive.ics.uci.edu/ml/datasets/Entree+Chicago+Recommendation+Data
Merchandise Recommendation:

**헬스케어 추천:**

Nursing Home - Provider Ratings Data Set http://data.medicare.gov/dataset/Nursing-Home-Compare-Provider-Ratings/mufm-vy8d

Hospital Ratings - Survey of Patients Hospital Experiences http://data.medicare.gov/dataset/Survey-of-Patients-Hospital-Experiences-HCAHPS-/rj76-22dk

**데이팅 추천:**

www.libimseti.cz - Dating website recommendation (collaborative filtering) http://www.occamslab.com/petricek/data/
Scholarly Paper Recommendation:

National University of Singapore - Scholarly Paper Recommendation http://www.comp.nus.edu.sg/~sugiyama/SchPaperRecData.html

# 참 잘했어요!